In [ ]:
import torch
import numpy as np
import tqdm
import pytorch_lightning as pl
from torch.utils.data import DataLoader
from pathlib import Path
from sklearn.model_selection import train_test_split

import IPython


# Import dei moduli che hai modificato
import sys
project_root = Path.cwd().parent

# 2. Aggiungilo a sys.path (convertendolo in stringa)
# abspath per sicurezza
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))
    print(f"Aggiunto al path: {project_root}")
sys.path.append('../src') # Assicurati che veda la cartella src
from src.data.dataset import PianoMmapDataset
from src.models.base_model import PianoGAN
from src.utils.tools import piano_roll_to_pretty_midi
from src.utils.visualization import display_prettymidi
# Set seed per riproducibilità
#pl.seed_everything(42)

REPO_ROOT = Path('.').resolve().parent
if REPO_ROOT not in sys.path:
    sys.path.insert(0, str(REPO_ROOT))


: 

In [ ]:
# 1. Configurazione Percorsi
DATA_DIR = REPO_ROOT / 'data'
MMAP_PREFIX = str(DATA_DIR / 'maestro') # Deve matchare il nome usato in preprocessing

# 2. Calcolo numero totale segmenti
# Un modo rapido è leggere la shape dal file _chords.dat che è piccolo
chords_path = f"{MMAP_PREFIX}_chords.dat"
if not Path(chords_path).exists():
    raise FileNotFoundError("Esegui prima main_preprocessing.ipynb!")

# I file .dat degli accordi sono (N, 8) uint8. 
# Dimensione file in byte = N * 8.
total_segments = Path(chords_path).stat().st_size // 8
print(f"Total segments found: {total_segments}")

# 3. Splitting (Copia esatta della logica di preprocessing)
all_indices = np.arange(total_segments)
train_idx, temp_idx = train_test_split(all_indices, test_size=0.2, random_state=42)
val_idx, test_idx = train_test_split(temp_idx, test_size=0.5, random_state=42)

print(f"Train: {len(train_idx)} | Val: {len(val_idx)} | Test: {len(test_idx)}")

# 4. Dataset & Dataloaders
# Nota: num_workers=0 su Windows per evitare problemi con memmap, altrimenti puoi aumentarlo
train_ds = PianoMmapDataset(MMAP_PREFIX, train_idx)
val_ds   = PianoMmapDataset(MMAP_PREFIX, val_idx)

train_loader = DataLoader(train_ds, batch_size=64, shuffle=True, num_workers=0, pin_memory=True)
val_loader   = DataLoader(val_ds,   batch_size=64, shuffle=False, num_workers=0)

In [ ]:
# Iperparametri
NOISE_DIM = 100
LR = 0.0002

# Istanza del modello
model = PianoGAN(noise_dim=NOISE_DIM, learning_rate=LR)

# (Opzionale) Stampa riassunto
#print(model)

In [ ]:
# Configurazione Checkpoint (salva il modello migliore)
checkpoint_callback = pl.callbacks.ModelCheckpoint(
    dirpath='../checkpoints',
    filename='pianogan-{epoch:02d}-{g_loss:.2f}',
    save_top_k=3,
    monitor='g_loss', # O un'altra metrica se la logghi
    mode='min',
)

trainer = pl.Trainer(
    accelerator='auto', # se lascio auto usa in automatico la GPU
    devices=1,
    max_epochs=10,
)

# Avvio Training
print("Starting Training...")
trainer.fit(model, train_loader, val_loader)

In [ ]:
# Rilevamento automatico del device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Se sei su un Mac con chip M1/M2/M3 (Apple Silicon), usa 'mps' per accelerazione hardware
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("Using Apple MPS (Metal Performance Shaders) acceleration!")
else:
    print(f"Using device: {device}")

# Sposta il modello sul device corretto
model.to(device)

# --- Ora procedi con la generazione ---
model.eval()

# Prepara i dati di test
prev, curr, chord = next(iter(val_loader))

# IMPORTANTE: Anche i tensori di input devono andare sullo stesso device!
prev = prev.to(device)
chord = chord.to(device)
noise = torch.randn(prev.size(0), 100, device=device)

# Genera
with torch.no_grad():
    generated_music = model(noise, prev, chord)

print("Generazione completata!")

# Visualize a sample from the DataLoader
# Estrai un batch

prev_bar, current_bar, chord_idx = next(iter(train_loader))

print("Previous bars shape:", prev_bar.shape)
print("Current bars shape:", current_bar.shape)
print("Chords batch shape:", chord_idx.shape)
print(f"Example chord index: {chord_idx[0].item()}") # Stampa l'indice dell'accordo del primo campione

# ho usato il codice di main_preprocessing per la visualizzazione

# forse andava normalizzato (lasciato al lettore per esercizio XD)
prev_pm = piano_roll_to_pretty_midi(prev_bar[1].squeeze().numpy()*100, fs=4)
display_prettymidi(
    prev_pm, 
    fs=4, 
    pitch_range=(24, 84),
    title=f"Visualization: previous_bar"
)
current_pm = piano_roll_to_pretty_midi(current_bar[1].squeeze().numpy()*100, fs=4)
display_prettymidi(
    current_pm, 
    fs=4, 
    pitch_range=(24, 84),
    title=f"Visualization: current_bar"
)

# Concatenate previous and current bars
concatenated_bars = torch.cat((prev_bar[1], current_bar[1]), dim=2)  # Concatenate along time axis
print("Concatenated bars shape:", concatenated_bars.shape)

concatenated_pm = piano_roll_to_pretty_midi(concatenated_bars.squeeze().numpy()*100, fs=4)
display_prettymidi(
    concatenated_pm, 
    fs=4, 
    pitch_range=(24, 84),
    title=f"Visualization: concatenated_bars"
)

audio = IPython.display.Audio(concatenated_pm.fluidsynth(fs=16000), rate=16000)
IPython.display.display(audio)